In [1]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

file_path = "/kaggle/input/credit/Creditcard_data.csv"
df = pd.read_csv(file_path)

print(df.head())
print("\nTarget Class Count:\n", df["Class"].value_counts())


/kaggle/input/credit/Creditcard_data.csv
   Time        V1        V2        V3        V4        V5        V6        V7  \
0     0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1     0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2     1 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3     1 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4     2 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.20

In [2]:
features = df.drop(columns=["Class"])
labels = df["Class"]

from sklearn.model_selection import train_test_split

X_tr, X_te, y_tr, y_te = train_test_split(
    features,
    labels,
    test_size=0.30,
    stratify=labels,
    random_state=42
)

In [3]:
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN, SMOTETomek

resamplers = [
    ("Random_Over", RandomOverSampler(random_state=42)),
    ("SMOTE", SMOTE(random_state=42)),
    ("Random_Under", RandomUnderSampler(random_state=42)),
    ("SMOTE_ENN", SMOTEENN(random_state=42)),
    ("SMOTE_Tomek", SMOTETomek(random_state=42))
]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

classifiers = [
    ("Logistic", LogisticRegression(max_iter=1200)),
    ("DecisionTree", DecisionTreeClassifier(random_state=42)),
    ("RandomForest", RandomForestClassifier(random_state=42)),
    ("NaiveBayes", GaussianNB()),
    ("SVM", SVC())
]

In [5]:
from sklearn.metrics import accuracy_score

performance = []

for samp_name, sampler in resamplers:
    X_bal, y_bal = sampler.fit_resample(X_tr, y_tr)

    for clf_name, clf in classifiers:
        clf.fit(X_bal, y_bal)
        predictions = clf.predict(X_te)

        acc = accuracy_score(y_te, predictions)
        performance.append([samp_name, clf_name, acc])

In [8]:
results_df = pd.DataFrame(
    performance,
    columns=["Sampling_Method", "Classifier", "Accuracy"]
)

print("\nAccuracy Comparison Across Models and Sampling Techniques:")
print(results_df.round(3)) 


Accuracy Comparison Across Models and Sampling Techniques:
   Sampling_Method    Classifier  Accuracy
0      Random_Over      Logistic     0.918
1      Random_Over  DecisionTree     0.970
2      Random_Over  RandomForest     0.991
3      Random_Over    NaiveBayes     0.953
4      Random_Over           SVM     0.875
5            SMOTE      Logistic     0.935
6            SMOTE  DecisionTree     0.987
7            SMOTE  RandomForest     0.987
8            SMOTE    NaiveBayes     0.953
9            SMOTE           SVM     0.435
10    Random_Under      Logistic     0.578
11    Random_Under  DecisionTree     0.388
12    Random_Under  RandomForest     0.668
13    Random_Under    NaiveBayes     0.621
14    Random_Under           SVM     0.746
15       SMOTE_ENN      Logistic     0.935
16       SMOTE_ENN  DecisionTree     0.875
17       SMOTE_ENN  RandomForest     0.983
18       SMOTE_ENN    NaiveBayes     0.944
19       SMOTE_ENN           SVM     0.362
20     SMOTE_Tomek      Logistic     